# Explicación del Código para Calcular la Matriz Origen-Destino Usando OSRM

Este script utiliza OSRM (Open Source Routing Machine) para calcular matrices de tiempo de viaje entre múltiples ubicaciones, tanto para *drive* como para *walk*. A continuación, se explican las partes clave del código y cómo funciona.

#### 1. Librerías Importadas

- `geopandas`: Usada para trabajar con datos geoespaciales en formato de DataFrame.
- `pandas`: Usada para manipular y organizar datos.
- `numpy`: Usada para cálculos numéricos y creación de matrices.
- `concurrent.futures`: Usada para realizar procesamiento paralelo, lo que acelera el cálculo de la matriz de tiempos de viaje.
- `requests`: Usada para hacer solicitudes HTTP a la API de OSRM.

#### 2. Función Principal: `estimate_origin_destination_matrix`

Esta función calcula una matriz de tiempos de viaje entre un conjunto de orígenes y destinos.

#### Proceso:
1. **Copiar Celdas**: Se hacen copias de los datos de celdas, ya que cada celda servirá tanto como origen como destino.
2. **Transformación de Coordenadas**: Se transforman las coordenadas al sistema local (EPSG: 32719) y luego de vuelta a WGS84 (EPSG: 4326). Esto asegura que los puntos de origen y destino estén en el formato correcto para OSRM.
3. **Extracción de Coordenadas**: Se extraen las coordenadas (longitud y latitud) de los puntos de origen y destino.
4. **Cálculo de la Matriz de Tiempos**: Se llama a la función `calculate_parallel_distance_matrix` para obtener la matriz de tiempos entre cada par de origen-destino.

#### 3. Función: `calculate_parallel_distance_matrix`

Esta función divide las listas de coordenadas en fragmentos más pequeños y realiza el cálculo en paralelo para mejorar la eficiencia.

#### Proceso:
1. **Inicializar Matriz**: Se crea una matriz vacía con valores iniciales de infinito, representando distancias no alcanzables.
2. **División en Fragmentos**: Se divide la lista de coordenadas de origen y destino en fragmentos más pequeños para no sobrecargar la API de OSRM.
3. **Procesamiento Paralelo**: Se utilizan hilos de ejecución paralelos para hacer solicitudes a la API de OSRM para cada fragmento de origen-destino.
4. **Construcción de la Matriz**: Los resultados de los fragmentos se combinan para construir la matriz completa.

#### 4. Función: `get_osrm_matrix`

Esta función realiza una solicitud HTTP a la API de OSRM para obtener los tiempos de viaje entre los conjuntos de coordenadas de origen y destino.

#### Proceso:
1. **Preparar la Solicitud**: Se preparan las coordenadas en el formato adecuado y se definen los índices de origen y destino.
2. **Hacer la Solicitud**: Se envía una solicitud HTTP a la API de OSRM usando el modo seleccionado (*drive* o *walk*).
3. **Procesar la Respuesta**: Si la solicitud es exitosa, se devuelve una matriz con los tiempos de viaje en minutos. Si hay un error, se retorna una matriz con valores de infinito.

#### 5. Función: `split_list`

Esta pequeña función divide una lista en fragmentos más pequeños para hacer más eficientes las solicitudes a la API.

#### 6. Uso del Código

Para ejecutar este código, se lee un archivo GeoJSON que contiene las celdas geográficas de interés. Se utilizan las siguientes líneas:

```python
celdas = gpd.read_file('celdas.geojson')
distancias_drive = estimate_origin_destination_matrix(celdas, 'drive')
distancias_walk = estimate_origin_destination_matrix(celdas, 'walk')
```

#### Explicación:
- **`celdas.geojson`**: Es un archivo que contiene las ubicaciones geográficas de interés.
- **`estimate_origin_destination_matrix(celdas, 'drive')`**: Calcula la matriz de tiempos de viaje para *drive*.
- **`estimate_origin_destination_matrix(celdas, 'walk')`**: Calcula la matriz de tiempos de viaje para *walk*.

#### 7. Resultado

Este código devolverá dos matrices de tiempos de viaje:
- **`distancias_drive`**: Tiempos de viaje en minutos entre todas las ubicaciones en auto.
- **`distancias_walk`**: Tiempos de viaje en minutos entre todas las ubicaciones a pie.

Ambas matrices están indexadas por las celdas de origen y muestran los tiempos de viaje a cada destino.


In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

import concurrent.futures
import requests

In [2]:
def estimate_origin_destination_matrix(cells, mode):
    gdf_origins = cells.copy()
    gdf_destinations = cells.copy()

    # Transform coordinates to local system and back to WGS84
    gdf_origins = gdf_origins.to_crs(32719)
    gdf_origins.geometry = gdf_origins.geometry.centroid
    gdf_origins = gdf_origins.to_crs(4326)

    gdf_destinations = gdf_destinations.to_crs(32719)
    gdf_destinations.geometry = gdf_destinations.geometry.centroid
    gdf_destinations = gdf_destinations.to_crs(4326)

    # Get coordinates of origins and destinations
    origin_coords = [(point.x, point.y) for point in gdf_origins.geometry]
    destination_coords = [(point.x, point.y) for point in gdf_destinations.geometry]

    # Calculate the distance matrix
    distance_matrix = calculate_parallel_distance_matrix(origin_coords, destination_coords, mode, max_size=1000)
    distance_matrix = pd.DataFrame(distance_matrix, index=gdf_origins.index)

    return distance_matrix

def calculate_parallel_distance_matrix(origins, destinations, mode, max_size=1000):
    # Initialize matrix with infinity (unreachable distances)
    matrix = np.full((len(origins), len(destinations)), np.inf)

    tasks = []
    for i, origin_chunk in enumerate(split_list(origins, max_size)):
        for j, destination_chunk in enumerate(split_list(destinations, max_size)):
            tasks.append((i, j, origin_chunk, destination_chunk))

    # Run tasks in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_task = {executor.submit(get_osrm_matrix, task[2], task[3], mode): task for task in tasks}
        for future in concurrent.futures.as_completed(future_to_task):
            task = future_to_task[future]
            i, j, origin_chunk, destination_chunk = task
            sub_matrix = future.result()

            if sub_matrix is not None:
                start_i = i * max_size
                end_i = start_i + len(origin_chunk)
                start_j = j * max_size
                end_j = start_j + len(destination_chunk)

                matrix[start_i:end_i, start_j:end_j] = sub_matrix

    return matrix

def get_osrm_matrix(origins, destinations, mode):
    coordinates = origins + destinations
    coordinates_str = ';'.join([f"{lon},{lat}" for lon, lat in coordinates])

    origin_indices = ';'.join([str(i) for i in range(len(origins))])
    destination_indices = ';'.join([str(i) for i in range(len(origins), len(coordinates))])

    # Define URL based on mode
    if mode == 'drive':
        url = f"http://localhost:5001/table/v1/driving/{coordinates_str}"
    elif mode == 'walk':
        url = f"http://localhost:5002/table/v1/walking/{coordinates_str}"

    params = {'sources': origin_indices, 'destinations': destination_indices, 'annotations': 'duration'}

    # Request to OSRM API
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        durations_seconds = np.array(data['durations'])
        durations_minutes = durations_seconds / 60
        return durations_minutes
    else:
        print(f"Error in OSRM request: {response.status_code}")
        return np.full((len(origins), len(destinations)), np.inf)

def split_list(coords, max_size):
    for i in range(0, len(coords), max_size):
        yield coords[i:i + max_size]

In [3]:
celdas = gpd.read_file('celdas.geojson')
distancias_drive = estimate_origin_destination_matrix(celdas, 'drive')
distancias_walk = estimate_origin_destination_matrix(celdas, 'walk')

In [4]:
distancias_drive

,0,1,2,3,4,5,6,7,8,9,...,647,648,649,650,651,652,653,654,655,656
0,0.000000,5.530000,0.585000,1.170000,3.640000,2.850000,8.600000,8.976667,7.998333,8.440000,...,20.538333,19.466667,21.183333,21.418333,20.651667,19.503333,20.430000,19.510000,19.985000,21.640000
1,0.388333,0.000000,0.973333,1.558333,4.028333,3.238333,8.988333,9.365000,8.386667,8.828333,...,20.926667,19.855000,21.571667,21.806667,21.040000,19.891667,20.818333,19.898333,20.373333,22.028333
2,5.333333,4.945000,0.000000,0.585000,3.055000,2.265000,8.015000,8.391667,7.413333,7.855000,...,19.953333,18.881667,20.598333,20.833333,20.066667,18.918333,19.845000,18.925000,19.400000,21.055000
3,4.748333,4.360000,5.333333,0.000000,2.470000,1.680000,7.430000,7.806667,6.828333,7.270000,...,19.368333,18.296667,20.013333,20.248333,19.481667,18.333333,19.260000,18.340000,18.815000,20.470000
4,6.221667,5.833333,6.806667,7.391667,0.000000,2.251667,8.903333,9.280000,8.301667,8.743333,...,21.353333,20.281667,21.998333,22.233333,21.466667,20.318333,21.245000,20.325000,20.800000,22.455000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,23.108333,22.720000,23.693333,24.278333,21.740000,20.950000,25.790000,26.166667,25.188333,25.630000,...,1.825000,1.686667,3.296667,1.915000,1.806667,0.000000,1.716667,1.220000,0.481667,2.136667
653,23.915000,23.526667,24.500000,25.085000,22.546667,21.756667,26.596667,26.973333,25.995000,26.436667,...,0.953333,1.443333,2.425000,3.676667,2.168333,1.761667,0.000000,0.876667,2.243333,3.898333
654,23.038333,22.650000,23.623333,24.208333,21.670000,20.880000,25.720000,26.096667,25.118333,25.560000,...,1.028333,0.755000,2.365000,3.130000,1.385000,1.215000,0.920000,0.000000,1.696667,3.351667
655,23.590000,23.201667,24.175000,24.760000,22.221667,21.431667,26.271667,26.648333,25.670000,26.111667,...,2.306667,2.168333,3.778333,1.433333,2.288333,0.481667,2.198333,1.701667,0.000000,1.655000


In [5]:
distancias_walk

,0,1,2,3,4,5,6,7,8,9,...,647,648,649,650,651,652,653,654,655,656
0,0.000000,15.485000,48.588333,59.283333,61.300000,62.830000,52.216667,49.065000,45.101667,53.955000,...,264.590000,243.943333,251.540000,253.950000,246.595000,242.623333,260.481667,243.986667,247.241667,255.766667
1,15.485000,0.000000,48.943333,59.638333,61.655000,63.185000,36.731667,33.580000,29.616667,39.223333,...,264.945000,244.298333,251.895000,254.305000,246.950000,242.978333,260.836667,244.341667,247.596667,256.121667
2,48.588333,48.943333,0.000000,10.695000,12.711667,14.241667,63.668333,74.248333,69.353333,54.336667,...,219.475000,198.828333,206.425000,208.948333,201.593333,197.621667,215.366667,198.985000,202.240000,210.765000
3,59.283333,59.638333,10.695000,0.000000,4.706667,3.583333,73.511667,84.091667,79.196667,64.180000,...,208.793333,188.146667,195.743333,198.266667,190.911667,186.940000,204.685000,188.303333,191.558333,200.083333
4,61.300000,61.655000,12.711667,4.706667,0.000000,8.166667,68.805000,79.385000,74.490000,59.473333,...,213.400000,192.753333,200.350000,202.873333,195.518333,191.546667,209.291667,192.910000,196.165000,204.690000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,242.623333,242.978333,197.621667,186.940000,191.546667,188.473333,244.246667,254.826667,249.931667,234.915000,...,30.513333,10.883333,17.463333,11.326667,8.581667,0.000000,26.405000,6.590000,9.918333,13.143333
653,260.481667,260.836667,215.366667,204.685000,209.291667,206.218333,262.105000,272.685000,267.790000,252.773333,...,4.108333,16.538333,9.301667,37.731667,24.580000,26.405000,0.000000,20.446667,31.811667,39.548333
654,243.986667,244.341667,198.985000,188.303333,192.910000,189.836667,245.610000,256.190000,251.295000,236.278333,...,24.555000,4.925000,11.505000,17.916667,4.513333,6.590000,20.446667,0.000000,12.091667,19.733333
655,247.241667,247.596667,202.240000,191.558333,196.165000,193.091667,248.865000,259.445000,254.550000,239.533333,...,35.920000,16.290000,22.870000,21.245000,14.083333,9.918333,31.811667,12.091667,0.000000,23.061667
